In [ ]:
# CountVectorizer, TfIdfVectorizer

In [ ]:
# !pip install Korpora
# !pip install tokenizers
# !pip install transformers

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Korpora import Korpora

from tokenizers import ByteLevelBPETokenizer, BertWordPieceTokenizer
from transformers import BertTokenizer, GPT2Tokenizer

In [ ]:
nsmc = Korpora.load('nsmc', force_download=True)

In [10]:
nsmc.train.texts[:5], nsmc.test.texts[:5]

(('아 더빙.. 진짜 짜증나네요 목소리',
  '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  '너무재밓었다그래서보는것을추천한다',
  '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'),
 ('굳 ㅋ',
  'GDNTOPCLASSINTHECLUB',
  '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
  '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
  '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'))

In [15]:
import os
import shutil

if os.path.exists('data-files/nsmc'):
    shutil.rmtree('data-files/nsmc')

os.mkdir('data-files/nsmc')

with open('data-files/nsmc/train.txt', 'w', encoding='utf-8') as f:
    for line in nsmc.train.get_all_texts():
        f.write(f"{line}\n")
with open('data-files/nsmc/test.txt', 'w', encoding='utf-8') as f:
    for line in nsmc.test.get_all_texts():
        f.write(f"{line}\n")

In [16]:
if os.path.exists('data-files/nsmc/wordpiece'):
    shutil.rmtree('data-files/nsmc/wordpiece')
os.mkdir('data-files/nsmc/wordpiece')

wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    ['data-files/nsmc/train.txt', 'data-files/nsmc/test.txt'],
    vocab_size=10000
) # 단어사전 학습

wordpiece_tokenizer.save_model('data-files/nsmc/wordpiece')

['data-files/nsmc/wordpiece\\vocab.txt']

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("data-files/nsmc/wordpiece", do_lower_case=False)

print(nsmc.train.texts[:5])
[bert_tokenizer.tokenize(sentence) for sentence in nsmc.train.texts[:5]]

In [24]:
result = bert_tokenizer(
    nsmc.train.texts[:5],
    padding='max_length',
    max_length=12,
    truncation=True
)

In [28]:
print( result.keys() )
result['input_ids'] # 단어 식별 번호로 구성된 문장 목록
result['token_type_ids'] # 단어가 포함된 문서의 순서 번호
result['attention_mask'] # 단어가 포함된 문서의 순서 번호

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]